In [1]:
# uncomment preferred option
#!pip install -q git+https://github.com/deepmipt/bert.git@feat/keras deeppavlov tensorflow-gpu==1.14.0
#!pip install -q git+https://github.com/deepmipt/bert.git@feat/keras deeppavlov tensorflow-gpu==2.0.0b1

In [2]:
import tensorflow as tf
import numpy as np

v = str(tf.__version__).split('.')[:2]
assert int(v[0]) > 1 or (int(v[1]) >= 13), 'this notebook was tested with TF v1.14.0 and v2.0.0-beta1'
tf.compat.v1.enable_eager_execution()
is_eager = tf.compat.v1.executing_eagerly()
is_eager

True

In [3]:
from bert_dp import bert
from bert_dp.metrics import F1Score

## Paraphrases data

In [18]:
BATCH_SIZE = 64  # both for training and evaluation for now
MAX_LEN = 128  #!= 52  125 for multi # the longest sequence in the paraphrase dataset, workaroud that make shape of all batch tensors equal
NUM_TRAIN_SAMPLES = 6656  # total of 7202 samples minus ~500 (actually 546) samples for validation
DYNAMIC_SEQ_LEN = True  # whether to pad each batch to the maximum length in the batch or in the whole dataset
NUM_CLASSES = 2  # binary classification as multiclass
DROP_REMAINDER = True # whether to for batches of fixed size

In [5]:
tf.keras.utils.get_file(
    fname='paraphraser.zip',
    origin='http://files.deeppavlov.ai/datasets/paraphraser.zip',
    cache_subdir='dataset',
    extract=True,
    cache_dir='.'
)
tf.keras.utils.get_file(
    fname='paraphraser_gold.zip',
    origin='http://files.deeppavlov.ai/datasets/paraphraser_gold.zip',
    cache_subdir='dataset',
    extract=True,
    archive_format='zip',
    cache_dir='.'
)
tf.keras.utils.get_file(
    fname='multi_cased_L-12_H-768_A-12.tar.gz',
    origin='https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip',
    cache_subdir='models',
    extract=True,
    cache_dir='.'
)

'./models/multi_cased_L-12_H-768_A-12.tar.gz'

In [6]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()


def data_gen(train_data=True):
    from deeppavlov.dataset_readers.paraphraser_reader import ParaphraserReader
    from bert_dp.tokenization import FullTokenizer
    ds = ParaphraserReader().read(data_path='dataset', do_lower_case=False)
    tokenizer = FullTokenizer(vocab_file='models/multi_cased_L-12_H-768_A-12/vocab.txt', do_lower_case=False)
    tds = ds['train'] if train_data else ds['test']
    for pair, label in tds:
        t1 = tokenizer.tokenize(pair[0])
        t2 = tokenizer.tokenize(pair[1])
        _truncate_seq_pair(t1, t2, 512)
        s = ['[CLS]'] + t1 + ['[SEP]'] + t2 + ['[SEP]']
        yield tokenizer.convert_tokens_to_ids(s), label

In [19]:
x_train_, y_train_, x_test_, y_test_ = [], [], [], []
for x, y in data_gen():
    x_train_.append(np.pad(x, pad_width=(0, MAX_LEN - len(x)), mode='constant'))
    y_train_.append(y)
for x, y in data_gen(train_data=False):
    x_test_.append(np.pad(x, pad_width=(0, MAX_LEN - len(x)), mode='constant'))
    y_test_.append(y)
x_train = np.array(x_train_)
y_train = np.array(y_train_)
x_test = np.array(x_test_)
y_test = np.array(y_test_)
len(x_train_), len(x_test_)

(7202, 1899)

In [20]:
# train_ds_ = tf.data.Dataset.from_generator(data_gen,
#                                            output_types=(tf.int32, tf.int32),
#                                            output_shapes=(tf.TensorShape([None]),
#                                                           tf.TensorShape([])))
# train_ds = train_ds_.take(NUM_TRAIN_SAMPLES)
# valid_ds = train_ds_.skip(NUM_TRAIN_SAMPLES)
# batched_train_ds = train_ds.padded_batch(batch_size=BATCH_SIZE,
#                                          padded_shapes=([-1] if DYNAMIC_SEQ_LEN else MAX_LEN,
#                                                         ()),
#                                          drop_remainder=DROP_REMAINDER)
# batched_valid_ds = valid_ds.padded_batch(batch_size=BATCH_SIZE,
#                                          padded_shapes=([-1] if DYNAMIC_SEQ_LEN else MAX_LEN,
#                                                         ()),
#                                          drop_remainder=DROP_REMAINDER)
# test_ds = tf.data.Dataset.from_generator(data_gen,
#                                          output_types=(tf.int32, tf.int32),
#                                          output_shapes=(tf.TensorShape([None]),
#                                                         tf.TensorShape([])),
#                                          args=(False,))
# batched_test_ds = test_ds.padded_batch(batch_size=BATCH_SIZE,
#                                        padded_shapes=([-1] if DYNAMIC_SEQ_LEN else MAX_LEN,
#                                                       ()),
#                                        drop_remainder=DROP_REMAINDER)

# tf.keras.Model construction and compilation for evaluation

In [21]:
try:
    del paraphrase_detector
    tf.keras.backend.clear_session()
    print('Session cleared')
except NameError:
    pass
paraphrase_detector = tf.keras.Sequential([bert.BERT(name='bert'),
                                           tf.keras.layers.Dropout(rate=0.5),
                                           tf.keras.layers.Dense(1)])

W0621 16:10:35.170635 139696497428288 deprecation_wrapper.py:119] From /home/nab/PycharmProjects/work/bert/bert_dp/embeddings.py:63: The name tf.keras.initializers.TruncatedNormal is deprecated. Please use tf.compat.v1.keras.initializers.TruncatedNormal instead.



In [22]:
# # some data to pass through model in order to construct it
try:
    for s in batched_train_ds:
        r = paraphrase_detector(s[0])
        print('Model constructed by means of passing one batch from tf.data')
        break
except NameError:
    paraphrase_detector.build(input_shape=(BATCH_SIZE, None if DYNAMIC_SEQ_LEN else MAX_LEN))
    print('Model constructed using .build() method')
    r1 = paraphrase_detector.predict(x_train[:BATCH_SIZE])

Model constructed using .build() method


In [23]:
NUM_EPOCHS = 5
# VAL_EVERY_N_BATCHES = 52  # ~50, but better suited for paraphraser dataset
# TRAIN_STEPS_PER_EPOCH = NUM_TRAIN_SAMPLES // BATCH_SIZE
# VALIDATION_STEPS = 512 // BATCH_SIZE  # calculated by hand for now (~546 samples with droppping remainder)
LEARNING_RATE = 2e-05
# learning_rate = tf.keras.optimizers.schedules.PolynomialDecay(initial_learning_rate=LEARNING_RATE,
#                                                               # n. of batches per epoch * ~NUM_EPOCHS for convergence 
#                                                               decay_steps=TRAIN_STEPS_PER_EPOCH * NUM_EPOCHS,
#                                                               end_learning_rate=1e-06)
# from bert_dp.weight_decay_optimizers import AdamW
paraphrase_detector.compile(#optimizer=AdamW(weight_decay=0.01,
#                                             learning_rate=LEARNING_RATE,
#                                             epsilon=1e-6),  # as in bert_dp
                            optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE,
                                                               epsilon=1e-6),
                            loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                            metrics=[tf.keras.metrics.BinaryAccuracy(),
                                     F1Score(num_classes=2, average='micro')
                                    ])

W0621 16:10:55.830227 139696497428288 deprecation.py:506] From /home/nab/PycharmProjects/work/venv/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:507: calling count_nonzero (from tensorflow.python.ops.math_ops) with axis is deprecated and will be removed in a future version.
Instructions for updating:
reduction_indices is deprecated, use axis instead


In [24]:
# baseline to ensure something is learning
# paraphrase_detector.evaluate(batched_test_ds)
paraphrase_detector.evaluate(x=x_test, y=y_test, batch_size=BATCH_SIZE)

1899/1899 [==============================] - 15s 8ms/sample - loss: 6.1007 - binary_accuracy: 0.5993 - f1_score: 0.6089


[6.100684211654623, 0.5992628, 0.6088817]

In [25]:
paraphraser_checkpoint_path = 'models/multi_cased_L-12_H-768_A-12/bert_model.ckpt'
paraphraser_saver = tf.compat.v1.train.Saver(var_list=paraphrase_detector.variables[:-2])  # loading without dense head
paraphraser_saver.restore(sess=None if is_eager else tf.keras.backend.get_session(),
                          save_path=paraphraser_checkpoint_path)

W0621 16:11:21.589987 139696497428288 saver.py:795] Saver is deprecated, please switch to tf.train.Checkpoint or tf.keras.Model.save_weights for training checkpoints. When executing eagerly variables do not necessarily have unique names, and so the variable.name-based lookups Saver performs are error-prone.
W0621 16:11:21.591475 139696497428288 deprecation.py:323] From /home/nab/PycharmProjects/work/venv/lib/python3.6/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [26]:
r2 = paraphrase_detector.predict(x_train[:BATCH_SIZE])
try:
    np.testing.assert_allclose(r1, r2)
except AssertionError:
    print('Outputs are changed')

Outputs are changed


In [27]:
# baseline to ensure something is learning
# paraphrase_detector.evaluate(batched_test_ds)
paraphrase_detector.evaluate(x=x_test, y=y_test, batch_size=BATCH_SIZE)

1899/1899 [==============================] - 13s 7ms/sample - loss: 9.2437 - binary_accuracy: 0.4007 - f1_score: 0.5485


[9.243731933621874, 0.40073723, 0.54849166]

# Training

In [28]:
history = paraphrase_detector.fit(x=x_train,
                                  y=y_train,
                                  epochs=NUM_EPOCHS,
                                  callbacks=[tf.keras.callbacks.TensorBoard(batch_size=BATCH_SIZE),
                                             tf.keras.callbacks.ReduceLROnPlateau(monitor='val_f1_score',
                                                                                  factor=0.5,
                                                                                  patience=3,
                                                                                  min_lr=1e-06)],
                                  validation_split=0.071)  # ~512 samples
# history = paraphrase_detector.fit(x=batched_train_ds.repeat(),
#                                   y=None,
#                                   epochs=NUM_EPOCHS * TRAIN_STEPS_PER_EPOCH // VAL_EVERY_N_BATCHES,  # workaround for more frequent evaluation
#                                   callbacks=[tf.keras.callbacks.TensorBoard(batch_size=BATCH_SIZE),
#                                              tf.keras.callbacks.ReduceLROnPlateau(monitor='val_sparse_binary_f1_score',
#                                                                                   factor=0.5,
#                                                                                   patience=3,
#                                                                                   min_lr=1e-06)],
#                                   validation_data=batched_valid_ds,
#                                   steps_per_epoch=VAL_EVERY_N_BATCHES,
#                                   validation_steps=VALIDATION_STEPS)

Train on 6690 samples, validate on 512 samples


W0621 16:11:55.465690 139696497428288 deprecation.py:323] From /home/nab/PycharmProjects/work/venv/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5
6690/6690 [==============================] - 250s 37ms/sample - loss: 1.3651 - binary_accuracy: 0.7197 - f1_score: 0.7229 - val_loss: 0.5434 - val_binary_accuracy: 0.6973 - val_f1_score: 0.7646
Epoch 2/5
6690/6690 [==============================] - 236s 35ms/sample - loss: 0.9406 - binary_accuracy: 0.7299 - f1_score: 0.7500 - val_loss: 0.5291 - val_binary_accuracy: 0.8066 - val_f1_score: 0.8084
Epoch 3/5
6690/6690 [==============================] - 237s 35ms/sample - loss: 0.8200 - binary_accuracy: 0.7937 - f1_score: 0.7297 - val_loss: 0.4448 - val_binary_accuracy: 0.7852 - val_f1_score: 0.8194
Epoch 4/5
6690/6690 [==============================] - 237s 35ms/sample - loss: 5.5630 - binary_accuracy: 0.5945 - f1_score: 0.6174 - val_loss: 9.0983 - val_binary_accuracy: 0.4102 - val_f1_score: 0.5412
Epoch 5/5
6690/6690 [==============================] - 238s 36ms/sample - loss: 9.9766 - binary_accuracy: 0.3532 - f1_score: 0.5640 - val_loss: 9.0983 - val_binary_accuracy: 0.4102 - v

In [29]:
# paraphrase_detector.evaluate(batched_test_ds)
metrics = paraphrase_detector.evaluate(x=x_test, y=y_test, batch_size=BATCH_SIZE)

1899/1899 [==============================] - 13s 7ms/sample - loss: 9.2436 - binary_accuracy: 0.4007 - f1_score: 0.5451
